<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;

public interface IInvoiceService
{
    void AddLine(LineItem lineItem);
    void RemoveLine(LineItem lineItem);
    void DisplayInvoice();
    void CalculateTotal();
}

public class InvoiceService : IInvoiceService
{
    private readonly Invoice _invoice;

    public InvoiceService(Invoice invoice)
    {
        _invoice = invoice;
    }

    public void AddLine(LineItem lineItem)
    {
        if (_invoice is IAddLine addLineInvoice)
        {
            addLineInvoice.AddLine(lineItem);
        }
    }

    public void RemoveLine(LineItem lineItem)
    {
        if (_invoice is IRemoveLine removeLineInvoice)
        {
            removeLineInvoice.RemoveLine(lineItem);
        }
    }

    public void DisplayInvoice()
    {
        _invoice.DisplayLineItems();
    }

    public void CalculateTotal()
    {
        _invoice.CalculateTotal();
    }
}

// Остальной код остается без изменений
public class LineItem
{
    public string Description { get; set; }
    public int Price { get; set; }
    public int Quantity { get; set; }

    public LineItem(string description, int price, int quantity)
    {
        Description = description;
        Price = price;
        Quantity = quantity;
    }

    public decimal GetTotal()
    {
        return Price * Quantity;
    }
}

public interface IAddLine
{
    void AddLine(LineItem lineItem);
}

public interface IRemoveLine
{
    void RemoveLine(LineItem lineItem);
}

public abstract class Invoice
{
    public string Description { get; set; }
    public string InvoiceNumber { get; set; }
    public DateTime IssueDate { get; set; }
    public decimal TotalAmount { get; protected set; }

    protected List<LineItem> _lineItems;

    protected Invoice(string invoiceNumber, DateTime issueDate, List<LineItem> lineItems)
    {
        InvoiceNumber = invoiceNumber;
        IssueDate = issueDate;
        _lineItems = lineItems ?? new List<LineItem>();
    }

    public virtual void CalculateTotal()
    {
        TotalAmount = 0;
        foreach (var item in _lineItems)
        {
            TotalAmount += item.GetTotal();
        }
    }

    public decimal GetTotal()
    {
        CalculateTotal();
        return TotalAmount;
    }

    public void DisplayLineItems()
    {
        Console.WriteLine($"Накладная №{InvoiceNumber}, Дата выдачи: {IssueDate.ToShortDateString()}");
        foreach (var item in _lineItems)
        {
            Console.WriteLine($"  - Позиция: {item.Description}, Цена: {item.Price}, Количество: {item.Quantity}, Итого: {item.GetTotal()}");
        }
    }
}

class GoodInvoice : Invoice, IAddLine, IRemoveLine
{
    public DateTime SupplyDate { get; set; }
    public bool ReturnAllowed { get; set; }

    public GoodInvoice(string invoiceNumber, DateTime issueDate, DateTime supplyDate, bool returnAllowed, List<LineItem> lineItems)
        : base(invoiceNumber, issueDate, lineItems)
    {
        SupplyDate = supplyDate;
        ReturnAllowed = returnAllowed;
    }

    void IAddLine.AddLine(LineItem lineItem)
    {
        _lineItems.Add(lineItem);
        Console.WriteLine($"Товар добавлен: {lineItem.Description}, Дата поставки: {SupplyDate.ToShortDateString()}");
    }

    void IRemoveLine.RemoveLine(LineItem lineItem)
    {
        _lineItems.Remove(lineItem);
        Console.WriteLine($"Товар удален: {lineItem.Description}");
    }

    public override void CalculateTotal()
    {
        base.CalculateTotal();
        Console.WriteLine(ReturnAllowed ? "Возврат разрешен." : "Возврат запрещен.");
        Console.WriteLine($"Общая сумма по товарной накладной: {TotalAmount}");
    }
}

class ServiceInvoice : Invoice, IAddLine, IRemoveLine
{
    public DateTime ServiceDate { get; set; }
    public bool ReturnAllowed { get; set; }

    public ServiceInvoice(string invoiceNumber, DateTime issueDate, DateTime serviceDate, bool returnAllowed, List<LineItem> lineItems)
        : base(invoiceNumber, issueDate, lineItems)
    {
        ServiceDate = serviceDate;
        ReturnAllowed = returnAllowed;
    }

    void IAddLine.AddLine(LineItem lineItem)
    {
        _lineItems.Add(lineItem);
        Console.WriteLine($"Услуга добавлена: {lineItem.Description}");
    }

    void IRemoveLine.RemoveLine(LineItem lineItem)
    {
        _lineItems.Remove(lineItem);
        Console.WriteLine($"Услуга: {lineItem.Description} была аннулирована.\nПричина: ошибка заказа.");
    }

    public override void CalculateTotal()
    {
        base.CalculateTotal();
        Console.WriteLine(ReturnAllowed ? "Возврат разрешен." : "Возврат запрещен.");
        Console.WriteLine($"Общая сумма по сервисной накладной: {TotalAmount}");
    }
}

class CombinedInvoice : Invoice, IAddLine, IRemoveLine
{
    public bool ReturnAllowed { get; set; }

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed, List<LineItem> lineItems)
        : base(invoiceNumber, issueDate, lineItems)
    {
        ReturnAllowed = returnAllowed;
    }

    void IAddLine.AddLine(LineItem lineItem)
    {
        _lineItems.Add(lineItem);
        Console.WriteLine($"Позиция добавлена: {lineItem.Description}");
    }

    void IRemoveLine.RemoveLine(LineItem lineItem)
    {
        _lineItems.Remove(lineItem);
        Console.WriteLine($"Позиция удалена: {lineItem.Description}");
    }

    public override void CalculateTotal()
    {
        base.CalculateTotal();
        Console.WriteLine(ReturnAllowed ? "Возврат разрешен." : "Возврат запрещен.");
        Console.WriteLine($"Общая сумма по комбинированной накладной: {TotalAmount}");
    }
}

// Использование с внедрением зависимостей
var lineItems = new List<LineItem>();
var goodInvoice = new GoodInvoice("INV001", DateTime.Now, DateTime.Now.AddDays(7), false, lineItems);
var goodInvoiceService = new InvoiceService(goodInvoice);

goodInvoiceService.AddLine(new LineItem("Товар 1", 300, 1));
goodInvoiceService.AddLine(new LineItem("Товар 2", 500, 2));
goodInvoiceService.DisplayInvoice();
goodInvoiceService.CalculateTotal();

Console.WriteLine("\n");

var serviceInvoice = new ServiceInvoice("INV002", DateTime.Now, DateTime.Now, true, new List<LineItem>());
var serviceInvoiceService = new InvoiceService(serviceInvoice);

serviceInvoiceService.AddLine(new LineItem("Услуга 1", 100, 1));
serviceInvoiceService.AddLine(new LineItem("Услуга 2", 50, 2));
serviceInvoiceService.DisplayInvoice();
serviceInvoiceService.CalculateTotal();

Console.WriteLine("\n");

IAddLine combinedInvoice = new CombinedInvoice("INV003", DateTime.Now, true, new List<LineItem>());
combinedInvoice.AddLine(new LineItem("Товар 3", 400, 2));
combinedInvoice.AddLine(new LineItem("Услуга 3", 200, 1));
((Invoice)combinedInvoice).DisplayLineItems();
((Invoice)combinedInvoice).CalculateTotal();

Товар добавлен: Товар 1, Дата поставки: 25.11.2024
Товар добавлен: Товар 2, Дата поставки: 25.11.2024
Накладная №INV001, Дата выдачи: 18.11.2024
  - Позиция: Товар 1, Цена: 300, Количество: 1, Итого: 300
  - Позиция: Товар 2, Цена: 500, Количество: 2, Итого: 1000
Возврат запрещен.
Общая сумма по товарной накладной: 1300


Услуга добавлена: Услуга 1
Услуга добавлена: Услуга 2
Накладная №INV002, Дата выдачи: 18.11.2024
  - Позиция: Услуга 1, Цена: 100, Количество: 1, Итого: 100
  - Позиция: Услуга 2, Цена: 50, Количество: 2, Итого: 100
Возврат разрешен.
Общая сумма по сервисной накладной: 200


Позиция добавлена: Товар 3
Позиция добавлена: Услуга 3
Накладная №INV003, Дата выдачи: 18.11.2024
  - Позиция: Товар 3, Цена: 400, Количество: 2, Итого: 800
  - Позиция: Услуга 3, Цена: 200, Количество: 1, Итого: 200
Возврат разрешен.
Общая сумма по комбинированной накладной: 1000
